In [1]:
import numpy as np
import os
from astropy.io import fits as pyfits
import shutil
import math
import PySimpleGUI as sg

In [2]:
def read_output(path_of_galfit_files,log_file,num_comp,num_runs):
    os.chdir(path_of_galfit_files)
    file = open(log_file,"r")
    lines = file.readlines()
    all_result_list = []
    all_error_list = []
    chars_to_remove = set(['(', ')', '\n','*',':',',','[',']','\t'])
    for run in range(num_runs): #start 1 = 7, end 1 = 20, start 2 = 32, end 2 = 45
        new_error_list = []
        new_result_list = []
        start = 7 + (2*num_comp + 11)*run
        end = start + 2 * num_comp
        for i in range(start,end):   
            line = lines[i]
            new_line = line.split(" ")
            final_line = [j for j in new_line if j]
            if((i - start) % 2 == 0): #if final line is a result line
                new_result_line = []
                for word in final_line:
                    new_word = ''.join([c for c in word if c not in chars_to_remove])
                    if(new_word == ""):
                         continue
                    new_result_line.append(new_word)
                new_result_list.append(new_result_line)
            else: #if final line is an error line
                new_error_line = []
                for word in final_line:
                    new_word = ''.join([c for c in word if c not in chars_to_remove])
                    if(new_word == ""):
                         continue
                    new_error_line.append(new_word)
                new_error_list.append(new_error_line)      
        all_result_list.append(new_result_list)
        all_error_list.append(new_error_list)
    modified_result_list = []
    for run in all_result_list:
        modified_result_run = []
        for component in run:
            modified_result_component = []
            comp_type = component[0] 
            comp_parameters = component[1:]
            for param in comp_parameters:
                modified_result_param = []
                n = param.find("e")
                if (n != -1):
                    base = param[0:n]
                    num_base = float(base)
                    dec = param[n+1:]
                    num_dec = float(dec)
                    tot_num = num_base * (10**num_dec)
                    modified_result_param.append(tot_num)
                else:
                    tot_num = float(param)
                    modified_result_param.append(tot_num)
                modified_result_component.append(modified_result_param[0])
            modified_result_run.append(modified_result_component)
        modified_result_list.append(modified_result_run)
    modified_error_list = []
    for run in all_error_list:
        modified_error_run = []
        for component in run:
            modified_error_component = []
            comp_type = component[0] 
            comp_parameters = component[0:]
            for param in comp_parameters:
                modified_error_param = []
                n = param.find("e")
                if (n != -1):
                    base = param[0:n]
                    num_base = float(base)
                    dec = param[n+1:]
                    num_dec = float(dec)
                    tot_num = num_base * (10**num_dec)
                    modified_error_param.append(tot_num)
                else:
                    tot_num = float(param)
                    modified_error_param.append(tot_num)
                modified_error_component.append(modified_error_param[0])
            modified_error_run.append(modified_error_component)
        modified_error_list.append(modified_error_run)
    return (modified_result_list,modified_error_list)

In [3]:
def analyze_output(result_list,error_list):
    num_run = len(result_list)
    all_result_list = []
    all_error_list = []
    for i in range(len(result_list[0])):
        comp_list = []
        for j in range(len(result_list[0][i])):
            new_list = []
            comp_list.append(new_list)
        all_result_list.append(comp_list)
    for i in range(num_run):
        for j in range(len(result_list[i])):
            for k in range(len(result_list[i][j])):
                all_result_list[j][k].append(result_list[i][j][k])
    for i in range(len(error_list[0])):
        comp_list = []
        for j in range(len(error_list[0][i])):
            new_list = []
            comp_list.append(new_list)
        all_error_list.append(comp_list)
    for i in range(num_run):
        for j in range(len(error_list[i])):
            for k in range(len(error_list[i][j])):
                all_error_list[j][k].append(error_list[i][j][k])   
    median_result_list = []
    for i in all_result_list:
        in_ar = []
        for j in i:
            med = np.median(j)
            in_ar.append(med)
        median_result_list.append(in_ar)
    std_dev_result_list = []
    for i in all_result_list:
        in_ar = []
        for j in i:
            stddev = np.std(j)
            in_ar.append(stddev)
        std_dev_result_list.append(in_ar)
    avg_result_list = []
    for i in all_result_list:
        in_ar = []
        for j in i:
            av = np.average(j)
            in_ar.append(av)
        avg_result_list.append(in_ar)
    avg_error_list = []
    for i in all_error_list:
        in_ar = []
        for j in i:
            av = np.average(j)
            in_ar.append(av)
        avg_error_list.append(in_ar) 
    return [avg_result_list, avg_error_list,median_result_list,std_dev_result_list]

In [4]:
def change_file(im,ext,sig_im,sig_ext,new_file_name,xstart,xend,ystart,yend): 
    hdul1 = pyfits.open(im)
    hdul2 = pyfits.open(sig_im)
    data1 = hdul1[ext].data
    data2 = hdul2[sig_ext].data
    new_pix_ar = []
    for i in range(ystart,yend):
        in_ar = []
        for j in range(xstart,xend):
            stddev_pix = data2[i][j]
            new_pix_val = np.random.normal(data1[i][j],stddev_pix)
            in_ar.append(new_pix_val)
        new_pix_ar.append(in_ar)    
    hdu = pyfits.PrimaryHDU(new_pix_ar)
    hdu.writeto("/Users/rohanvenkat/Desktop/" + new_file_name)

In [5]:
def run_galfit(n,galfit_path,store_output_im,num_comp,root_dir,desired_dir,feedme_file,im_file,im_ext,sig_file,sig_ext,xstart,xend,ystart,yend): 
    os.chdir(desired_dir) #moves to the desired directory
    new_dir = os.path.join(desired_dir,"galfit_files") #path to the storage folder inside the desired_directory
    path_feedme = os.path.join(root_dir,feedme_file) #path of the feedme file
    path_im = os.path.join(root_dir,im_file) #path of the im file
    galfit_dir = os.path.join(galfit_path,"galfit")
    path_sig = os.path.join(root_dir,sig_file)
    f = open("output_file.txt", "w")
    num_not_crashed = 0
    if(os.path.isfile(path_feedme)): 
        if(os.path.isdir(new_dir)):
            shutil.rmtree(new_dir)
        os.mkdir(os.path.join(new_dir)) #makes the folder which will store galfit files
        for i in range(n):
            print(i,end="\r")
            number = "0"
            if(i < 10):
                number = "0" + str(i + 1) 
            else:
                number = str(i)
            galfit_output = "galfit." + number
            change_file(path_im,im_ext,path_sig,sig_ext,"altered_im.fits",xstart,xend,ystart,yend)
            os.system(galfit_dir + " " + path_feedme) #runs galfit 
            if(os.path.isfile("galfit.01")):
                num_not_crashed += 1
            if(store_output_im == False):
                if(os.path.isfile("galfit.01")): #if this runs it means galfit crashed so no output was made
                    os.remove("galfit.01")
                    os.remove("output.fits")
                os.remove("altered_im.fits")
            else:
                os.rename("altered_im.fits",os.path.join(new_dir,"altered_im" + str(i) + ".fits"))
                os.rename("galfit.01",os.path.join(new_dir,galfit_output))
                os.rename("output.fits",os.path.join(new_dir,"output" + number + ".fits"))
        os.rename("fit.log",os.path.join(new_dir,"fit.log"))
        tup = read_output(new_dir,"fit.log",num_comp,num_not_crashed) 
        result = tup[0]
        error = tup[1]
        f.write("RESULTS\n")
        for iteration in result:
            for comp in iteration:
                for i in comp:
                    f.write(str('%.3f'%(i)))
                    f.write("\t")
                f.write("\n")
            f.write("\n")
        f.write("ERRORS\n")
        for iteration in error:
            for comp in iteration:
                for i in comp:
                    f.write(str('%.3f'%(i)))
                    f.write("\t")
                f.write("\n")
            f.write("\n")
        output_list = analyze_output(result,error)
        f.write("AVGRESULTS\n")
        for line in output_list[0]:
            for comp in line:
                f.write(str('%.3f'%(comp)))
                f.write("\t")
            f.write("\n")
        f.write("\n")
        f.write("AVGERRORS\n")
        for line in output_list[1]:
            for comp in line:
                f.write(str('%.3f'%(comp)))
                f.write("\t")
            f.write("\n")
        f.write("\n")
        f.write("MEDIANRESULTS\n")
        for line in output_list[2]:
            for comp in line:
                f.write(str('%.3f'%(comp)))
                f.write("\t")
            f.write("\n")
        f.write("\n")
        f.write("STDDEV\n")
        for line in output_list[3]:
            for comp in line:
                f.write(str('%.3f'%(comp)))
                f.write("\t")
            f.write("\n")
        os.rename(os.path.join(desired_dir,"output_file.txt"),os.path.join(new_dir,"output_file.txt"))
        return
    else:    
        exit()

In [10]:
def main(): #run_galfit(im_file,im_ext,sig_file,sig_ext,xstart,xend,ystart,yend): 
    arg0 = [sg.Text('Number of iterations'),sg.InputText()]    
    arg1 = [sg.Text("Directory of galfit executable"),sg.FolderBrowse(key="galfit_ex")]
    arg2 = [sg.Text('Store altered images?'),sg.Combo(['True','False'])]
    arg3 = [sg.Text("Number of galaxies in feedme file"),sg.InputText()]
    arg4 = [sg.Text("Directory of feedme file"),sg.FolderBrowse(key="feedme")]
    arg5 = [sg.Text("Directory to output results"),sg.FolderBrowse(key="output")]
    arg6 = [sg.Text("Name of feedme file"),sg.InputText()]
    arg7 = [sg.Text("Name of image file"),sg.InputText()]
    arg8 = [sg.Text("Extension of input file"),sg.InputText()]
    arg9 = [sg.Text("Name of sigma file"),sg.InputText()]
    arg10 = [sg.Text("Extension of sigma file"),sg.InputText()]
    arg11 = [sg.Text("Xstart"),sg.InputText()]
    arg12 = [sg.Text("Xend"),sg.InputText()]
    arg13 = [sg.Text("Ystart"),sg.InputText()]
    arg14 = [sg.Text("Yend"),sg.InputText()]
    final = [sg.Submit(), sg.Cancel()]
    layout = [arg0,arg1,arg2,arg3,arg4,arg5,arg6,arg7,arg8,arg9,arg10,arg11,arg12,arg13,arg14,final]
    window = sg.Window('Window Title', layout)    
    event, values = window.read()    
    print(values)
    window.close()
    run_galfit(int(values[0]),str(values["galfit_ex"]),bool(values[1]),int(values[2]),str(values["feedme"]),str(values["output"]),str(values[3]),str(values[4]),int(values[5]),str(values[6]),int(values[7]),int(values[8]),int(values[9]),int(values[10]),int(values[11]))